In [1]:
# расскоментируйте код ниже, чтобы установить все зависимости
# !pip install -q tensorboard==2.13.0 \
#     tensorflow==2.13.0 \
#     pyarrow==12.0.1 \
#     polars==0.18.6 \
#     tqdm==4.65.0 \
#     scipy==1.10.1 \
#     scikit-learn==1.3.0 \
#     numpy==1.24.3 \
#     Pillow==10.0.0

In [2]:
import os
import numpy as np
import pandas as pd
import polars as pl
from tqdm import tqdm

from typing import List

import numpy as np
import scipy.sparse as sp
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer

from PIL import Image
import requests
import matplotlib.pyplot as plt

In [3]:
movies_df = pl.read_parquet('imdb_movies.parquet')
movies_df

title_id,poster_url,type,name,original_name,description,genre,date,rating_count,rating_value,keywords,featured_review,stars,directors,creators
str,str,str,str,str,str,list[str],str,f64,f64,list[str],str,list[str],list[str],list[str]
"""tt0111161""","""https://m.medi…","""Movie""","""Побег из Шоуше…","""The Shawshank …","""Over the cours…","[""Drama""]","""1994-10-14""",2.755277e6,9.3,"[""prison"", ""based on the works of stephen king"", … ""voice over narration""]","""It is no wonde…","[""Tim Robbins"", ""Morgan Freeman"", ""Bob Gunton""]","[""Frank Darabont""]","[""Stephen King"", ""Frank Darabont""]"
"""tt0068646""","""https://m.medi…","""Movie""","""Крестный отец""","""The Godfather""","""Don Vito Corle…","[""Crime"", ""Drama""]","""1972-03-24""",1.917037e6,9.2,"[""mafia"", ""patriarch"", … ""rise to power""]","""I love this mo…","[""Marlon Brando"", ""Al Pacino"", ""James Caan""]","[""Francis Ford Coppola""]","[""Mario Puzo"", ""Francis Ford Coppola""]"
"""tt0468569""","""https://m.medi…","""Movie""","""Тёмный рыцарь""","""The Dark Knigh…","""When the menac…","[""Action"", ""Crime"", ""Drama""]","""2008-08-14""",2.728169e6,9.0,"[""dc comics"", ""psychopath"", … ""clown""]","""If someone els…","[""Christian Bale"", ""Heath Ledger"", ""Aaron Eckhart""]","[""Christopher Nolan""]","[""Jonathan Nolan"", ""Christopher Nolan"", ""David S. Goyer""]"
"""tt0050083""","""https://m.medi…","""Movie""","""12 разгневанны…","""12 Angry Men""","""The jury in a …","[""Crime"", ""Drama""]","""1961-11-13""",816234.0,9.0,"[""jury"", ""dialogue driven"", … ""jury room""]","""&apos;12 Angry…","[""Henry Fonda"", ""Lee J. Cobb"", ""Martin Balsam""]","[""Sidney Lumet""]","[""Reginald Rose""]"
"""tt0071562""","""https://m.medi…","""Movie""","""Крестный отец …","""The Godfather …","""The early life…","[""Crime"", ""Drama""]","""1974-12-18""",1.305122e6,9.0,"[""revenge"", ""1950s"", … ""lake tahoe nevada""]","""The original G…","[""Al Pacino"", ""Robert De Niro"", ""Robert Duvall""]","[""Francis Ford Coppola""]","[""Francis Ford Coppola"", ""Mario Puzo""]"
"""tt0108052""","""https://m.medi…","""Movie""","""Список Шиндлер…","""Schindler&apos…","""In German-occu…","[""Biography"", ""Drama"", ""History""]","""1994-05-21""",1.388917e6,9.0,"[""holocaust"", ""nazi"", … ""german""]","""This Movie was…","[""Liam Neeson"", ""Ralph Fiennes"", ""Ben Kingsley""]","[""Steven Spielberg""]","[""Thomas Keneally"", ""Steven Zaillian""]"
"""tt0167260""","""https://m.medi…","""Movie""","""Властелин коле…","""The Lord of th…","""Gandalf and Ar…","[""Action"", ""Adventure"", ""Drama""]","""2004-01-22""",1.892435e6,9.0,"[""journey"", ""orc"", … ""ring""]","""Wonderful on e…","[""Elijah Wood"", ""Viggo Mortensen"", ""Ian McKellen""]","[""Peter Jackson""]","[""J.R.R. Tolkien"", ""Fran Walsh"", ""Philippa Boyens""]"
"""tt0110912""","""https://m.medi…","""Movie""","""Криминальное ч…","""Pulp Fiction""","""The lives of t…","[""Crime"", ""Drama""]","""1995-09-29""",2.115956e6,8.9,"[""nonlinear timeline"", ""overdose"", … ""drug dealing""]","""I can only spe…","[""John Travolta"", ""Uma Thurman"", ""Samuel L. Jackson""]","[""Quentin Tarantino""]","[""Quentin Tarantino"", ""Roger Avary""]"
"""tt0120737""","""https://m.medi…","""Movie""","""Властелин коле…","""The Lord of th…","""A meek Hobbit …","[""Action"", ""Adventure"", ""Drama""]","""2002-03-01""",1.920951e6,8.8,"[""ring"", ""quest"", … ""elf""]","""Here is one fi…","[""Elijah Wood"", ""Ian McKellen"", ""Orlando Bloom""]","[""Peter Jackson""]","[""J.R.R. Tolkien"", ""Fran Walsh"", ""Philippa Boyens""]"


In [4]:
movies_df = movies_df.with_columns(
    # далее будем работать с признаками как со списком значений
    pl.col("type").apply(lambda x: [x]).alias("type"),
    # создадим колонку date_years_4, которая содержит диапазон лет, по которым мы будем искать похожие тайтлы
    pl.col("date")
    .apply(lambda x: [str(v) for v in range(int(x[:4]) - 4, int(x[:4]) + 4 + 1)])
    .alias("date_years_4"),
)

## Описание данных

Для решения задачи вам доступны следующие данные из файла `imdb_movies.parquet`

| Поле            | Тип                  | Описание                                 |
|---              |---                   |---                                       |
| title_id        | str                  | IMDb идентификатор                       |
| poster_url      | str                  | Ссылка на постер                         |
| type            | str                  | Тип тайтла: Movie или TVSeries           |
| name            | str                  | Название тайтла на русском               |
| original_name   | str                  | Оригинальное название тайтла             |
| description     | str                  | Описание тайтла с IMDb                   |
| genre           | list[str]            | Список жанров                            |
| date            | str                  | Дата выпуска тайтла                      |
| rating_count    | int                  | Количество отзывов для тайтла            |
| rating_value    | float                | Средний рейтинг тайтла                   |
| keywords        | list[str]            | Ключевые слова для тайтла (сгенерированы пользователями IMDb) |
| featured_review | str                  | Избранная рецензия для тайтла            |
| stars           | list[str]            | Список ключевых актеров                  |
| directors       | list[str]            | Список режиссеров                        |
| creators        | list[str]            | Список создателей                        |

## Оценивание

В качестве метрики качества используется hitrate@10, которую можно интерпретировать как **вероятность, что хотя бы один из топ-10 рекомендуемых объектов является релевантным**. Чтобы получить максимальный балл, достаточно добиться hitrate@10 = 0.3

В качестве `y_relevant` используется тайтлы, которые встречаются вместе в оценках пользователей

In [5]:
TOP_K = 10
SUBMISSION_PATH = "submission.parquet"
RELEVANT_TITLES_PATH = "relevant_titles_subsample.parquet"


def hitrate(y_relevant: List[str], y_preds: List[str], k: int = TOP_K) -> int:
    return int(len(set(y_relevant).intersection(y_preds[:k])) > 0)


# код для подсчета метрики качества
def print_score():
    hitrate_list = []
    # загрузим предсказания в словарь title -> [title_id]
    user_preds = {
        title_id: recs for title_id, recs in pl.read_parquet(SUBMISSION_PATH).rows()
    }
    for title_id, relevant_items in pl.read_parquet(RELEVANT_TITLES_PATH).rows():
        # обрежем рекомендации до top-K значений
        recommended_titles = user_preds.get(title_id, [])[:TOP_K]
        hitrate_list.append(hitrate(relevant_items, recommended_titles))

    mean_hitrate = float(np.mean(hitrate_list))
    print(f"HITRATE@10 = {mean_hitrate}")

## Построим рекомендации на основе категориальных признаков

По каждому признаку сначала построим разреженную матрицу $A$, где $A_{i, j} = 1$, если признак $j$ относится к тайтлу $i$

После чего построим матрицу похожести с помощью cosine метрики расстояния

In [6]:
similarities_by_col = {}
feature_columns = [
    "date_years_4",
    "type",
    "keywords",
    "genre",
    "directors",
    "creators",
    "stars",
]

for col in tqdm(feature_columns):
    # закодируем категориальные признаки
    mapping = {k: v for v, k in enumerate(movies_df[col].explode().unique().to_list())}

    # построим разреженную матрицу признаков для каждого тайтла
    rows = []
    cols = []
    values = []
    for row_ind, value in enumerate(movies_df[col]):
        value = [] if value is None else value

        # колонки – значения признака
        col_inds = [mapping[x] for x in value]
        rows.extend([row_ind] * len(col_inds))
        # ставим 1, если такой признак относится к тайтлу
        values.extend([1] * len(col_inds))
        cols.extend(col_inds)

    # построим матрицу похожести для каждого признака с помощью матрики косинусного расстояния
    sparse_data = sp.csr_matrix((values, (rows, cols)))
    sparse_data = normalize(sparse_data, norm="l2", axis=1)
    similarities_by_col[col] = (sparse_data @ sparse_data.T).A

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:04<00:00,  1.69it/s]


## Добавим текстовые признаки

Закодируем их самым простым способом, с помощью tf-idf

Можно было бы воспользоваться более продвинутыми способами вроде w2v или openai эмбеддингами, но даже такое простое решение позволяет учитывает одинаковые слова в двух отзывах или описаниях тайтлов

In [7]:
# построим по уни- и би-граммам признаки на основе текстовых данных и матрицу похожести с метрикой cosine distance
vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), norm='l2')
sparse_data = vectorizer.fit_transform(movies_df['featured_review'].to_list())
similarities_by_col['featured_review'] = (sparse_data @ sparse_data.T).A

vectorizer = TfidfVectorizer(sublinear_tf=True, ngram_range=(1, 2), norm='l2')
sparse_data = vectorizer.fit_transform(movies_df['description'].to_list())
similarities_by_col['description'] = (sparse_data @ sparse_data.T).A

## Построим рекомендации в виде ансамбля

Для этого просуммируем все похожести с весами, то есть $sim(title_i, title_j) = \sum_k sim_k(title_i, title_j)$, где $sim_k$ – полученная матрица похожести для признака $k$

In [8]:
N = len(movies_df)

similarities = np.zeros((N, N))
for k, v in tqdm(similarities_by_col.items()):
    weight = 1  # каждый признак имеет одинаковую важность
    # итоговая похожесть -- усреднение похожести по каждому признаку
    similarities += weight * (v - np.eye(N))  # вычитаем 

similarities.shape

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:08<00:00,  1.10it/s]


(9351, 9351)

In [9]:
def get_recommendations(title_ind: int, k: int = 10):
    # для рекомендации похожих тайтлов к title_ind, отсортируем похожести
    # similarities[title_ind] по убыванию и возьмем первые k индексов
    nearest_inds = np.argsort(similarities[title_ind])[::-1][:k]
    return movies_df[nearest_inds]["title_id"].to_list()


TOP_K = 10

submission = []
for title_ind in tqdm(range(len(movies_df))):
    title_id = movies_df["title_id"][title_ind]
    recommended_titles = get_recommendations(title_ind, TOP_K)
    submission.append((title_id, recommended_titles))

pl.DataFrame(submission, schema=("title_id", "recs")).write_parquet(
    "submission.parquet"
)
print_score()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 9351/9351 [00:07<00:00, 1320.27it/s]


HITRATE@10 = 0.308


🚀🚀🚀

## Добавим усреднение с учетом скора по признакам

In [12]:
# построим только по одному признаку рекомендации и посчитаем скор
# попробуйте сами оценить веса с помощью отложенной выборки relevant_titles_subsample
feature_to_score = {
    'date_years_4': ...,
    'type': ...,
    'keywords': ...,
    'genre': ...,
    'directors': ...,
    'creators': ...,
    'stars': ...,
    'featured_review': ...,
    'description': ...,
}

In [13]:
N = len(movies_df)

similarities = np.zeros((N, N))
for k, v in tqdm(similarities_by_col.items()):
    weight = feature_to_score[k]  # используем скор признака как его вес в итоговом ансамбле
    similarities += weight * (v - np.eye(N))

similarities.shape

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:09<00:00,  1.09s/it]


(9351, 9351)

In [14]:
def get_recommendations(title_ind: int, k: int = 10):
    nearest_inds = np.argsort(similarities[title_ind])[::-1][:k]
    return movies_df[nearest_inds]['title_id'].to_list()


TOP_K = 10

submission = []
for title_ind in tqdm(range(len(movies_df))):
    title_id = movies_df['title_id'][title_ind]
    recommended_titles = get_recommendations(title_ind, TOP_K)
    submission.append((title_id, recommended_titles))
    
pl.DataFrame(submission, schema=('title_id', 'recs')).write_parquet('submission.parquet')
print_score()

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 9351/9351 [00:06<00:00, 1522.37it/s]


HITRATE@10 = 0.318
